In [92]:
import pandas as pd
import numpy as np
from google_play_scraper import app, Sort, reviews_all
import emoji
from collections import Counter, OrderedDict
import regex
import string
from sklearn.model_selection import train_test_split
import nltk
import json
from keras.preprocessing.sequence import pad_sequences

In [40]:
def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.EMOJI_DATA)


def split_count(text):
    data = regex.findall(r'\X',text)
    return Counter(data)

def remove_punc(st):
    st = st.lower()
    p ="".join([i for i in st if i not in string.punctuation])
    return p

def remove_nonEng(st):
    return " ".join(w for w in nltk.wordpunct_tokenize(st) \
         if w.lower() in set(nltk.corpus.words.words()) or not w.isalpha())

def replace_emojis(st):
    dEmo = {
    "great ": ['❤','😍','👌','🔥','🥰','😎', '🤩','😘','💯','😊',
              '♥','👏','⭐','💛','🙏','💪','💙','💖','💕', '❣',
              '💓','🌟', '🎯','🖤','✌','💞','✨','😇', '💝','🤘',
              '💗','💚','🤟','🤍','🙌🏽','👏🏽','🐐','👑','🥇','💜','💟'],
    
    "good ": ['👍','🎮','⚽','🆗','👍🏻','👌🏽','👌🏿','👌🏾','💪','💪🏽','👍🏽'],
    
    "bad ": ['👎','😠','🙂','💔','😐','😔','😕','🙂','😤','😑','🤦','🤦🏻','🙅'],
    
    "horrible ": ['🤮','🖕','🤬','😡','😡','💩','👎🏼','👎🏾','🖕🏻','🖕🏼','❌']     
    }

    
    st = regex.findall(r'\X',st)
    for i in range(len(st)):
        if st[i] in emoji.EMOJI_DATA:
            flag=True
            for j in dEmo:            
                if st[i] in dEmo[j]:
                    st[i] = j
                    flag=False
                    
            if flag : st[i] ="UNK "                                           
    return "".join([i for i in st]).split()

def remove_UNK(sentence):
    return [i for i in sentence if i != "UNK"]
    

def remove_UNK_from_set(st):
    for i in range(len(st)):
        st[i] = remove_UNK(st[i])
    return st
def remove_empty(st):
    return [x for x in st if x]
def join_set(st,y):
    new_set = []
    for i,j in zip(st,y):
        if j == 1 : new_set.append([" ".join(i),"good"])
        else : new_set.append([" ".join(i),"bad"])
    return new_set

In [8]:
Score_hero_reviews = reviews_all(
    'com.firsttouchgames.hero2',
    sleep_milliseconds=0, 
    lang='en', 
    country='us',
    sort=Sort.NEWEST, 
)



In [9]:
herodf = pd.DataFrame(np.array(Score_hero_reviews),columns=['review'])


herodf = herodf.join(pd.DataFrame(herodf.pop('review').tolist()))



herodf.head()


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,dd7b38cb-1d13-489e-9ed9-fbfd027080a5,Elizabeth Afumah,https://play-lh.googleusercontent.com/a/ALm5wu...,The game is great 😂😂,5,0,None,2022-11-17 08:18:53,None,NaT
1,c4597cbc-91c6-4fbc-8aae-6965bdeecd3d,samaira rana,https://play-lh.googleusercontent.com/a/ALm5wu...,nice,5,0,2.60,2022-11-17 07:34:13,None,NaT
2,269299f8-25e6-49cf-b9f0-dd4f78db6f48,Mavhungu Lutendo,https://play-lh.googleusercontent.com/a-/ACNPE...,good game,4,0,2.60,2022-11-17 06:10:37,None,NaT
3,ba9129d4-f028-4f9d-8687-434eea2cd903,Wai Yai,https://play-lh.googleusercontent.com/a/ALm5wu...,very good,5,0,2.60,2022-11-17 06:10:14,None,NaT
4,5246ea2c-82c4-408c-aa4c-b5a9ee4921c4,Sumit Tanti,https://play-lh.googleusercontent.com/a/ALm5wu...,sumi t tanti,5,0,2.60,2022-11-17 05:52:49,None,NaT


In [78]:
g,c,b =0,0,0
for i in herodf['score']:
    if i>3:
        g+=1
    elif i <3 :
        c+=1
    else : b+=1
    
print(g,c,b)     

29548 9753 3629


In [6]:
PVPreviews = reviews_all(
    'com.firsttouchgames.smp',
    sleep_milliseconds=0, 
    lang='en', 
    country='us',
    sort=Sort.NEWEST, 
)

In [7]:
PVPdf = pd.DataFrame(np.array(PVPreviews),columns=['review'])


PVPdf = PVPdf.join(pd.DataFrame(PVPdf.pop('review').tolist()))



PVPdf.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,c78cbb5f-1575-4f10-bd24-3fb92291cd04,Steven Dawit,https://play-lh.googleusercontent.com/a/ALm5wu...,nice,5,0,None,2022-11-17 12:18:18,None,NaT
1,17de2066-f6dc-4102-a2c8-3919b7efba8e,ALAMIN KHAN,https://play-lh.googleusercontent.com/a-/ACNPE...,not a bad,2,0,2.41,2022-11-17 11:42:09,None,NaT
2,bbdef483-f58f-40fd-b57c-d1c0cf2a85c8,Abs Nayeem,https://play-lh.googleusercontent.com/a/ALm5wu...,"Pls, make an option to customize the players",5,0,None,2022-11-17 11:32:31,None,NaT
3,1e607e71-9851-4645-ad5b-005e5d22337d,Satyam Singh,https://play-lh.googleusercontent.com/a/ALm5wu...,The only thing that sucks is player's strength...,1,0,None,2022-11-17 11:12:11,None,NaT
4,0fcbf84c-9a9d-4aee-bc88-7ab177aaa6af,18 BT102 Goutham,https://play-lh.googleusercontent.com/a-/ACNPE...,very intelligent game,5,0,2.41,2022-11-17 10:09:50,None,NaT


In [3]:
herodf = pd.read_csv("../input/reviews-set/herodf.csv")
PVPdf = pd.read_csv("../input/reviews-set/PVP.csv")

df = herodf.append(PVPdf, ignore_index=True)
df.head()

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,0,dd7b38cb-1d13-489e-9ed9-fbfd027080a5,Elizabeth Afumah,https://play-lh.googleusercontent.com/a/ALm5wu...,The game is great 😂😂,5,0,NaN,2022-11-17 08:18:53,NaN,NaN
1,1,c4597cbc-91c6-4fbc-8aae-6965bdeecd3d,samaira rana,https://play-lh.googleusercontent.com/a/ALm5wu...,nice,5,0,2.6,2022-11-17 07:34:13,NaN,NaN
2,2,269299f8-25e6-49cf-b9f0-dd4f78db6f48,Mavhungu Lutendo,https://play-lh.googleusercontent.com/a-/ACNPE...,good game,4,0,2.6,2022-11-17 06:10:37,NaN,NaN
3,3,ba9129d4-f028-4f9d-8687-434eea2cd903,Wai Yai,https://play-lh.googleusercontent.com/a/ALm5wu...,very good,5,0,2.6,2022-11-17 06:10:14,NaN,NaN
4,4,5246ea2c-82c4-408c-aa4c-b5a9ee4921c4,Sumit Tanti,https://play-lh.googleusercontent.com/a/ALm5wu...,sumi t tanti,5,0,2.6,2022-11-17 05:52:49,NaN,NaN


In [4]:
print(len(df['score']))

152426


In [45]:
sat = []
dis = []

for (comment,score) in zip(df['content'],df['score']):
    if score>3 :
        sat.append(replace_emojis(remove_punc(str(comment))))
    else : 
        dis.append(replace_emojis(remove_punc(str(comment))))

In [46]:
sat = sat[:42000]

In [47]:
print(dis[1099])

['its', 'a', 'nice', 'game', 'but', 'needs', 'a', 'lot', 'of', 'improvements']


In [48]:
print(sat[0])

['the', 'game', 'is', 'great', 'UNK', 'UNK']


In [49]:
w = set(nltk.corpus.words.words())

for i in range(len(sat)):
    for j in range(len(sat[i])):
        s = regex.findall(r'\X',sat[i][j])
        flag = False
        for l in s :
            if l not in w and l not in emoji.EMOJI_DATA:
                flag = True
        if flag : sat[i][j] = "UNK"
    
            

In [50]:
for i in range(len(dis)):
    for j in range(len(dis[i])):
        s = regex.findall(r'\X',dis[i][j])
        flag = False
        for l in s :
            if l not in w and l not in emoji.EMOJI_DATA:
                flag = True
        if flag : dis[i][j] = "UNK"

In [51]:
print(df['content'][0])
print(sat[0])

The game is great 😂😂
['the', 'game', 'is', 'great', 'UNK', 'UNK']


In [52]:
print(len(sat),len(dis))
sat = remove_empty(remove_UNK_from_set(sat))
dis = remove_empty(remove_UNK_from_set(dis))
print(len(sat),len(dis))

42000 41840
39866 40784


In [62]:
with open("satS.txt","w") as f:
    f.write(json.dumps(sat))
with open("disS.txt","w") as f1:
    f1.write(json.dumps(dis))

In [71]:
satY = np.ones(len(sat),int)
disY = np.zeros(len(dis),int)

In [72]:
X = sat + dis
Y = np.concatenate((satY,disY))

In [55]:
Finaldf = pd.DataFrame(join_set(X,Y), columns=['review', 'sentiment'])

In [56]:
print(len(Finaldf["review"]))

80650


In [57]:
Finaldf.head()

,review,sentiment
0,the game is great,good
1,nice,good
2,good game,good
3,very good,good
4,sumi t tanti,good


In [63]:
Finaldf.to_csv('MyReviewsSet.csv')